### Sites
* [Rimi](https://www.rimi.lt/e-parduotuve/)
* [Iki](https://www.lastmile.lt/)
* [Maxima](https://barbora.lt)
* [Lidl](https://www.lidl.lt)

In [8]:
from items import candidates
from shopscraping import scrape, get_view
from IPython.display import Markdown, display
import os
import pandas as pd
from recipe_formatter import normalise_recipe, parse_recipe

def collect_catalogue(items=None, offline=False, balance_weights=True,
                      sorter={'by':['comparative_unit_discount_price', 'shop', 'discount'],
                              'ascending': [True, True, False]}):
    '''Return mapping (item -> df of scrape). 
    If offline is True: use results of history file. 
    If fix_weight is True: if some items have weights unbalanced, 
    apply weight value (this is an experimental option cus shops not always provide it right)
    
    Otherwise, scrape items given and update history file
    
    Input: list of items or None
    Output: mapping (item -> df of scrape) or None if there are some missing items in candidates
    ''' 
    item_dfs = dict()
    
    if items is None:
        d = candidates
    else:    
        d = {key: candidates[key] for key in items if key in candidates}
        missing_items = [key for key in items if key not in candidates]
        if missing_items:
            print('Please add urls for items:', missing_items)
            
    for item, item_urls in d.items():
        if not(offline):
            print(f'Item = {item.upper()} ... ', end=' ')
        item_dfs[item] = scrape(item_urls, offline=offline, sorter=False)
        units = item_dfs[item]['comparative_unit']
        units_vnt = [n for n in units if n in ['vnt']]
        units_weight = [n for n in units if n in ['kg', 'l']]
        unexpected_units = [n for n in units if n not in ['vnt', 'kg', 'l']]
        most_common_unit = max(set(units_weight), key=units_weight.count) if units_weight else 'vnt'
        if len(unexpected_units):
            print(f'Warning: unexpected units in item = {item}: {unexpected_units} wont balance')
        elif balance_weights:
            if units_vnt and units_weight:
                item_dfs[item].loc[units == 'vnt', 'comparative_unit_price'] = \
                item_dfs[item].loc[units == 'vnt', 'comparative_unit_price'] / item_dfs[item].loc[units == 'vnt', 'weight']
                item_dfs[item].loc[units == 'vnt', 'comparative_unit_discount_price'] = \
                item_dfs[item].loc[units == 'vnt', 'comparative_unit_discount_price'] / item_dfs[item].loc[units == 'vnt', 'weight']
                item_dfs[item].loc[units == 'vnt', 'comparative_unit'] = most_common_unit
        if not(offline):
            print('Done', end='\n')
        if sorter:
            item_dfs[item] = item_dfs[item].sort_values(**sorter)
    return item_dfs

def display_items(df, extra_columns=[], sorter=None):
    display(Markdown(get_view(df, extra_columns=extra_columns, sorter=sorter).to_markdown()))

def display_catalogue(cart, extra_columns=[], sorter=None):
    for item in cart:
        display_items(cart[item], extra_columns=extra_columns, sorter=sorter)

def investigate_recipe(recipe_data, offline=True):
    recipe_df = normalise_recipe(recipe_data)
    #display(recipe_df)
    catalogue = collect_catalogue(items=recipe_df['name'], offline=offline)
    if catalogue:
        prices, discount_prices, discounts = [], [], []
        for weight, unit, title in recipe_df.values:
            price = None
            discount_price = None
            discount = None
            
            if title not in catalogue:
                pass
                #print('Please update candidate file with:', title)

            elif unit is None:
                pass
                
            elif not(all(catalogue[title]['comparative_unit'] == unit)):
                print(f'Inconsistent weight units: [{title}] is in {unit} but catalogue contains:', 
                      catalogue[title]['comparative_unit'].tolist())
            else:
                #display_items(catalogue[title])
                price = weight * catalogue[title]['comparative_unit_price'].min()
                discount_price = weight * catalogue[title]['comparative_unit_discount_price'].min()
                discount = price - discount_price
            prices.append(price)
            discount_prices.append(discount_price)
            discounts.append(discount)
            
        recipe_df['price'] = [round(n, 2) if n else n for n in prices]
        recipe_df['discount_price'] = [round(n, 2) if n else n for n in discount_prices]
        recipe_df['discount'] = [round(n, 2) if n else n for n in discounts]
        return recipe_df



fridge_dir, fridge_file = 'fridge', 'fridge.csv'
fridge_file_path = os.path.join(fridge_dir, fridge_file)
df = pd.read_csv(fridge_file_path)

items_onwatch = sorted(candidates.keys())
items_own = df['items'].loc[df['own']==1].tolist()
items_hunt = df['items'].loc[df['hunt']==1].tolist()
items_asap = df['items'].loc[df['asap']==1].tolist()

print(f'onwatch:', items_onwatch, end='\n\n')
print(f'own:', items_own, end='\n\n')
print(f'hunt:', items_hunt, end='\n\n')
print(f'asap:', items_asap, end='\n\n')

onwatch: ['Avietės', 'Aviečių uogienė', 'Avižų gėrimas', 'Bananai', 'Bulvės', 'Burokėliai', 'Druska', 'Džiovinti krapai', 'Grietinė', 'Imbieras', 'Karis', 'Kefyras', 'Kiaušiniai', 'Kokosų gėrimas', 'Kopūstai', 'Laurų lapai', 'Maltos aitriosios paprikos', 'Morkos', 'Paprikos', 'Pienas UAT', 'Pomidorų pasta', 'Rausvieji svogūnai', 'Ridikėliai', 'Skumbrė', 'Sviestas', 'Sviestinis moliūgas', 'Svogūnai', 'Svogūnų laiškai', 'Trumpavaisiai agurkai', 'Viščiukų blauzdelės', 'Viščiukų filė', 'Šilauogės', 'Špinatai']

own: ['Bulvės', 'Burokėliai', 'Druska', 'Grietinė', 'Imbieras', 'Karis', 'Kokosų gėrimas', 'Kopūstai', 'Laurų lapai', 'Maltos aitriosios paprikos', 'Morkos', 'Paprikos', 'Pomidorų pasta', 'Rausvieji svogūnai', 'Ridikėliai', 'Sviestas', 'Sviestinis moliūgas', 'Svogūnai', 'Trumpavaisiai agurkai', 'Viščiukų blauzdelės', 'Špinatai']

hunt: ['Avietės', 'Aviečių uogienė', 'Avižų gėrimas', 'Bananai', 'Bulvės', 'Burokėliai', 'Džiovinti krapai', 'Grietinė', 'Kefyras', 'Kiaušiniai', 'Kokosų g

## Update your items
### All the items

In [6]:
items_onwatch

['Avietės',
 'Aviečių uogienė',
 'Avižų gėrimas',
 'Bananai',
 'Bulvės',
 'Burokėliai',
 'Druska',
 'Džiovinti krapai',
 'Grietinė',
 'Imbieras',
 'Karis',
 'Kefyras',
 'Kiaušiniai',
 'Kokosų gėrimas',
 'Kopūstai',
 'Laurų lapai',
 'Maltos aitriosios paprikos',
 'Morkos',
 'Paprikos',
 'Pienas UAT',
 'Pomidorų pasta',
 'Rausvieji svogūnai',
 'Ridikėliai',
 'Skumbrė',
 'Sviestas',
 'Sviestinis moliūgas',
 'Svogūnai',
 'Svogūnų laiškai',
 'Trumpavaisiai agurkai',
 'Viščiukų blauzdelės',
 'Viščiukų filė',
 'Šilauogės',
 'Špinatai']

In [9]:
catalogue = collect_catalogue(items=items_onwatch, offline=False)
display_catalogue(catalogue)

Item = AVIETĖS ...  RIMI warning: (no price) https://www.rimi.lt/e-parduotuve/lt/produktai/vaisiai-darzoves-ir-geles/vaisiai-ir-uogos/uogos/avietes-rimi-225g/p/988086
Done
Done = AVIEČIŲ UOGIENĖ ...  
Done = AVIŽŲ GĖRIMAS ...  
Done = BANANAI ...  
Done = BULVĖS ...  
Done = BUROKĖLIAI ...  
Done = DRUSKA ...  
Done = DŽIOVINTI KRAPAI ...  
Done = GRIETINĖ ...  
Done = IMBIERAS ...  
Done = KARIS ...  
Done = KEFYRAS ...  
Done = KIAUŠINIAI ...  
Done = KOKOSŲ GĖRIMAS ...  
Done = KOPŪSTAI ...  
Done = LAURŲ LAPAI ...  
Done = MALTOS AITRIOSIOS PAPRIKOS ...  
Done = MORKOS ...  
Done = PAPRIKOS ...  
Done = PIENAS UAT ...  
Done = POMIDORŲ PASTA ...  
Done = RAUSVIEJI SVOGŪNAI ...  
Done = RIDIKĖLIAI ...  
Done = SKUMBRĖ ...  
Done = SVIESTAS ...  
Done = SVIESTINIS MOLIŪGAS ...  
RIMI warning: (no price) https://www.rimi.lt/e-parduotuve/lt/produktai/vaisiai-darzoves-ir-geles/darzoves-ir-grybai/svogunai-porai-ir-cesnakai/svogunai-d-40-60mm-kg/p/937069
Done
RIMI warning: (no price) http

|    | shop   | title                                                                                                                                                                                           | off    |   price |   discount_price | price_per   | discount_per   |   weight | date       | valid_until   |
|---:|:-------|:------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|:-------|--------:|-----------------:|:------------|:---------------|---------:|:-----------|:--------------|
|  2 | MAXIMA | [Šaldytos avietės WELL DONE, 400 g](https://barbora.lt/produktai/sald-avietes-well-done-400-g-400-g)                                                                                            |        |    3.59 |             3.59 | 8.98€/kg    | 8.98€/kg       |    0.4   | 2025-08-24 |               |
|  7 | RIMI   | [Šaldytos avietės RIMI, 400 g](https://www.rimi.lt/e-parduotuve/lt/produktai/saldytas-maistas/saldytos-darzoves-vaisiai-ir-uogos/saldyti-vaisiai-ir-uogos/saldytos-avietes-rimi-400-g/p/811678) |        |    4.59 |             4.59 | 11.47€/kg   | 11.47€/kg      |    0.4   | 2025-08-24 |               |
|  3 | IKI    | [Avietės, 125 g](https://www.lastmile.lt/chain/IKI/product/Avietes-125-g-2457)                                                                                                                  | -20.1% |    2.49 |             1.99 | 19.92€/kg   | 15.92€/kg      |    0.13  | 2025-08-24 | 2025-08-24    |
|  0 | MAXIMA | [Avietės, 250 g](https://barbora.lt/produktai/avietes-250-g-13955)                                                                                                                              |        |    4.19 |             4.19 | 16.76€/kg   | 16.76€/kg      |    0.25  | 2025-08-24 |               |
|  6 | RIMI   | [Lietuviškos avietės, 1 kl. 125g](https://www.rimi.lt/e-parduotuve/lt/produktai/vaisiai-darzoves-ir-geles/vaisiai-ir-uogos/uogos/lietuviskos-avietes-1-kl-125g/p/903227)                        | -20.8% |    2.89 |             2.29 | 23.12€/kg   | 18.32€/kg      |    0.125 | 2025-08-24 | 2025-08-25    |
|  4 | IKI    | [Avietės, 250 g](https://www.lastmile.lt/chain/IKI/product/Avietes-250-g-84657)                                                                                                                 |        |    4.79 |             4.79 | 19.16€/kg   | 19.16€/kg      |    0.27  | 2025-08-24 |               |
|  1 | MAXIMA | [Avietės WELL DONE, 125 g](https://barbora.lt/produktai/avietes-well-done-125-g)                                                                                                                |        |    2.99 |             2.99 | 23.92€/kg   | 23.92€/kg      |    0.125 | 2025-08-24 |               |
|  5 | RIMI   | [Avietės RIMI, 125 g](https://www.rimi.lt/e-parduotuve/lt/produktai/vaisiai-darzoves-ir-geles/vaisiai-ir-uogos/uogos/avietes-rimi-125-g/p/904821)                                               |        |    2.99 |             2.99 | 23.92€/kg   | 23.92€/kg      |    0.125 | 2025-08-24 |               |

|    | shop   | title                                                                                                                                                                                               | off    |   price |   discount_price | price_per   | discount_per   |   weight | date       | valid_until   |
|---:|:-------|:----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|:-------|--------:|-----------------:|:------------|:---------------|---------:|:-----------|:--------------|
|  0 | MAXIMA | [Aviečių uogienė SKANOVĖ, 900 g](https://barbora.lt/produktai/avieciu-uogiene-skanove-900-g)                                                                                                        |        |    5.19 |             5.19 | 5.77€/kg    | 5.77€/kg       |    0.899 | 2025-08-24 |               |
|  9 | RIMI   | [Aviečių uogienė PŪRE, 420 g](https://www.rimi.lt/e-parduotuve/lt/produktai/bakaleja/konservuotas-maistas/uogienes-dzemai-ir-tyres/avieciu-uogiene-pure-420-g/p/156329)                             | -25.5% |    3.29 |             2.45 | 7.83€/kg    | 5.83€/kg       |    0.42  | 2025-08-24 | 2025-08-25    |
|  1 | MAXIMA | [Aviečių uogienė WELL DONE, 350 g](https://barbora.lt/produktai/avieciu-uogiene-well-done-350-g)                                                                                                    |        |    2.89 |             2.89 | 8.26€/kg    | 8.26€/kg       |    0.35  | 2025-08-24 |               |
|  2 | IKI    | [Aviečių uogienė PURE, 900 g](https://www.lastmile.lt/chain/PROMO-CashandCarry/product/Avieciu-uogiene-PURE-900-g-U262U07)                                                                          |        |    7.74 |             7.74 | 8.6€/kg     | 8.6€/kg        |    0.9   | 2025-08-24 |               |
|  8 | RIMI   | [Aviečių uogienė KĖDAINIŲ KONSERVAI, 430 g](https://www.rimi.lt/e-parduotuve/lt/produktai/bakaleja/konservuotas-maistas/uogienes-dzemai-ir-tyres/avieciu-uogiene-kedainiu-konservai-430-g/p/164482) |        |    3.89 |             3.89 | 9.05€/kg    | 9.05€/kg       |    0.43  | 2025-08-24 |               |
|  6 | IKI    | [Kėdainių aviečių uogienė, 430 g](https://www.lastmile.lt/chain/IKI/product/Kedainiu-avieciu-uogiene-430-g-240633)                                                                                  |        |    3.99 |             3.99 | 9.28€/kg    | 9.28€/kg       |    0.64  | 2025-08-24 |               |
|  3 | IKI    | [MOČIUTĖS aviečių uogienė, 600 g](https://www.lastmile.lt/chain/PROMO-CashandCarry/product/MOCIUTES-avieciu-uogiene-600-g-V26P778)                                                                  |        |    5.75 |             5.75 | 9.58€/kg    | 9.58€/kg       |    0.6   | 2025-08-24 |               |
|  7 | IKI    | [Tradicinė aviečių uogienė, 230 g](https://www.lastmile.lt/chain/GERU-MINCIU-uKIS/product/Tradicine-avieciu-uogiene-230-g-UO)                                                                       |        |    3.99 |             3.99 | 9.97€/kg    | 9.97€/kg       |    0.4   | 2025-08-24 | 2024-01-31    |
| 10 | RIMI   | [Aviečių uogienė SPILVA, 380 g](https://www.rimi.lt/e-parduotuve/lt/produktai/bakaleja/konservuotas-maistas/uogienes-dzemai-ir-tyres/avieciu-uogiene-spilva-380-g/p/182128)                         |        |    3.89 |             3.89 | 10.24€/kg   | 10.24€/kg      |    0.38  | 2025-08-24 |               |
|  4 | IKI    | [Aviečių uogienė KĖDAINIŲ, 430 g](https://www.lastmile.lt/chain/PROMO-CashandCarry/product/Avieciu-uogiene-KEDAINIU-430-g-V26P42H)                                                                  |        |    4.67 |             4.67 | 10.86€/kg   | 10.86€/kg      |    0.43  | 2025-08-24 |               |
|  5 | IKI    | [Aviečių uogienė VILROKA, 1,2 kg](https://www.lastmile.lt/chain/PROMO-CashandCarry/product/Avieciu-uogiene-VILROKA-12-kg-V26P818)                                                                   |        |    8.51 |             8.51 | 17.02€/kg   | 17.02€/kg      |    0.5   | 2025-08-24 |               |

|    | shop   | title                                                                                                                                                                                                                | off    |   price |   discount_price | price_per   | discount_per   |   weight | date       | valid_until   |
|---:|:-------|:---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|:-------|--------:|-----------------:|:------------|:---------------|---------:|:-----------|:--------------|
|  5 | IKI    | [Avižų augalinis gėrimas su kalciu ir vitaminais VALSOIA, 1 l](https://www.lastmile.lt/chain/IKI/product/Avizu-augalinis-gerimas-su-kalciu-ir-vitaminais-VALSOIA-1-l-247793)                                         | -49.8% |    2.99 |             1.5  | 2.99€/l     | 1.5€/l         |     1.04 | 2025-08-24 |               |
|  3 | MAXIMA | [Avižų gėrimas FARM MILK, 1 l](https://barbora.lt/produktai/avizu-gerimas-farm-milk-1-l)                                                                                                                             |        |    1.59 |             1.59 | 1.59€/l     | 1.59€/l        |     1    | 2025-08-24 |               |
|  8 | RIMI   | [Avižų gėrimas ICA, 1l](https://www.rimi.lt/e-parduotuve/lt/produktai/augaliniai-produktai/augaliniai-gerimai/augaliniai-gerimai/avizu-gerimas-ica-1l/p/812023)                                                      |        |    1.69 |             1.69 | 1.69€/l     | 1.69€/l        |     1    | 2025-08-24 |               |
|  7 | IKI    | [Ekologiškas avižų gėrimas MARGĖ, 1 l](https://www.lastmile.lt/chain/IKI/product/Ekologiskas-avizu-gerimas-MARGE-1-l-275941)                                                                                         |        |    2.29 |             2.29 | 2.29€/l     | 2.29€/l        |     1.04 | 2025-08-24 |               |
| 10 | RIMI   | [Avižų gėrimas YOOK  BARISTA, 1 l](https://www.rimi.lt/e-parduotuve/lt/produktai/augaliniai-produktai/augaliniai-gerimai/augaliniai-gerimai/avizu-gerimas-yook-barista-1-l/p/4000826)                                | -20.8% |    2.89 |             2.29 | 2.89€/l     | 2.29€/l        |     1    | 2025-08-24 | 2025-08-25    |
| 13 | RIMI   | [Avižų ir braškių - ban. sulčių gėr. CIDO, 1 l](https://www.rimi.lt/e-parduotuve/lt/produktai/augaliniai-produktai/augaliniai-gerimai/augaliniai-kavos-gerimai/avizu-ir-braskiu---ban-sulciu-ger-cido-1-l/p/1019843) |        |    2.29 |             2.29 | 2.29€/l     | 2.29€/l        |     1    | 2025-08-24 |               |
|  4 | MAXIMA | [Ekologiškas avižų gėrimas be pridėtinio cukraus WELL DONE, 1 l](https://barbora.lt/produktai/ekologiskas-avizu-gerimas-be-pridetinio-cukraus-well-done-1-l)                                                         |        |    2.49 |             2.49 | 2.49€/l     | 2.49€/l        |     1    | 2025-08-24 |               |
| 11 | RIMI   | [Augalinis avižų gėrimas VALSOIA, 1 l](https://www.rimi.lt/e-parduotuve/lt/produktai/augaliniai-produktai/augaliniai-gerimai/augaliniai-gerimai/augalinis-avizu-gerimas-valsoia-1-l/p/968463)                        |        |    2.89 |             2.89 | 2.89€/l     | 2.89€/l        |     1    | 2025-08-24 |               |
| 12 | RIMI   | [Avižų gėrimas TERE DEARY Barista, 1 l](https://www.rimi.lt/e-parduotuve/lt/produktai/augaliniai-produktai/augaliniai-gerimai/augaliniai-gerimai/avizu-gerimas-tere-deary-barista-1-l/p/994218)                      |        |    2.89 |             2.89 | 2.89€/l     | 2.89€/l        |     1    | 2025-08-24 |               |
|  1 | MAXIMA | [Avižų gėrimas NATURLI', 1 l](https://barbora.lt/produktai/avizu-gerimas-naturli-1-l)                                                                                                                                |        |    2.99 |             2.99 | 2.99€/l     | 2.99€/l        |     1    | 2025-08-24 |               |
|  9 | RIMI   | [Ekologiškas avižų gėrimas OATLY, 1 l](https://www.rimi.lt/e-parduotuve/lt/produktai/augaliniai-produktai/augaliniai-gerimai/augaliniai-gerimai/ekologiskas-avizu-gerimas-oatly-1-l/p/993125)                        |        |    2.99 |             2.99 | 2.99€/l     | 2.99€/l        |     1    | 2025-08-24 |               |
|  0 | MAXIMA | [Avižų gėrimas ALPRO, 1 l](https://barbora.lt/produktai/avizu-gerimas-alpro-1-l)                                                                                                                                     |        |    3.19 |             3.19 | 3.19€/l     | 3.19€/l        |     1    | 2025-08-24 |               |
|  6 | IKI    | [Avižų gėrimas ALPRO, 1l](https://www.lastmile.lt/chain/IKI/product/Avizu-gerimas-ALPRO-1l-241218)                                                                                                                   |        |    3.29 |             3.29 | 3.29€/l     | 3.29€/l        |     1.06 | 2025-08-24 |               |
|  2 | MAXIMA | [Avižų gėrimas ALPRO BARISTA, 1 l](https://barbora.lt/produktai/avizu-gerimas-alpro-barista-1-l)                                                                                                                     |        |    3.69 |             3.69 | 3.69€/l     | 3.69€/l        |     1    | 2025-08-24 |               |

|    | shop   | title                                                                                                                                                                   | off    |   price |   discount_price | price_per   | discount_per   |   weight | date       | valid_until   |
|---:|:-------|:------------------------------------------------------------------------------------------------------------------------------------------------------------------------|:-------|--------:|-----------------:|:------------|:---------------|---------:|:-----------|:--------------|
|  1 | IKI    | [Bananai BON VIA](https://www.lastmile.lt/chain/IKI/product/Bananai-BON-VIA-287376)                                                                                     | -28.8% |    1.39 |             0.99 | 1.39€/kg    | 0.99€/kg       |        1 | 2025-08-24 | 2025-08-24    |
|  0 | MAXIMA | [Bananai, nuo 20 cm, 1 kg](https://barbora.lt/produktai/bananai-1-kg)                                                                                                   |        |    1.39 |             1.39 | 1.39€/kg    | 1.39€/kg       |        1 | 2025-08-24 |               |
|  2 | RIMI   | [Bananai Cavendish, 20+cm , 1kg](https://www.rimi.lt/e-parduotuve/lt/produktai/vaisiai-darzoves-ir-geles/vaisiai-ir-uogos/bananai/bananai-cavendish-20-cm-1kg/p/270939) |        |    1.39 |             1.39 | 1.39€/kg    | 1.39€/kg       |        1 | 2025-08-24 |               |

...

|    | shop   | title                                                                                                                                                                                                      | off    |   price |   discount_price | price_per   | discount_per   |   weight | date       | valid_until   |
|---:|:-------|:-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|:-------|--------:|-----------------:|:------------|:---------------|---------:|:-----------|:--------------|
|  0 | MAXIMA | [Lietuviški trumpavaisiai agurkai, 1 kg](https://barbora.lt/produktai/lietuviski-trumpavaisiai-agurkai-1-kg)                                                                                               | -50.2% |    2.99 |             1.49 | 2.99€/kg    | 1.49€/kg       |      1   | 2025-08-24 | 2025-08-25    |
|  2 | RIMI   | [Lietuviški trumpavaisiai agurkai, kg](https://www.rimi.lt/e-parduotuve/lt/produktai/vaisiai-darzoves-ir-geles/darzoves-ir-grybai/agurkai/lietuviski-trumpavaisiai-agurkai-kg/p/245039)                    | -46.8% |    2.99 |             1.59 | 2.99€/kg    | 1.59€/kg       |      1   | 2025-08-24 | 2025-08-25    |
|  1 | IKI    | [Lietuviški trumpavaisiai agurkai ](https://www.lastmile.lt/chain/IKI/product/Lietuviski-trumpavaisiai-agurkai-2488)                                                                                       |        |    2.99 |             2.99 | 2.99€/kg    | 2.99€/kg       |      1   | 2025-08-24 |               |
|  3 | RIMI   | [Lietuviški fasuoti trumpavaisiai agurkai,500g](https://www.rimi.lt/e-parduotuve/lt/produktai/vaisiai-darzoves-ir-geles/darzoves-ir-grybai/agurkai/lietuviski-fasuoti-trumpavaisiai-agurkai-500g/p/946894) |        |    1.89 |             1.89 | 3.78€/kg    | 3.78€/kg       |      0.5 | 2025-08-24 |               |

|    | shop   | title                                                                                                                                                                                                                                                               | off    |   price |   discount_price | price_per   | discount_per   |   weight | date       | valid_until   |
|---:|:-------|:--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|:-------|--------:|-----------------:|:------------|:---------------|---------:|:-----------|:--------------|
|  6 | IKI    | [Viščiukų broilerių blauzdelės CLEVER](https://www.lastmile.lt/chain/IKI/product/Visciuku-broileriu-blauzdeles-CLEVER-287406)                                                                                                                                       |        |    3.29 |             3.29 | 3.29€/kg    | 3.29€/kg       |     1.02 | 2025-08-24 |               |
|  2 | MAXIMA | [Šviežios viščiukų broilerių blauzdelės WELL DONE, 1 kg](https://barbora.lt/produktai/sviezios-visciuku-broileriu-blauzdeles-well-done-1-kg)                                                                                                                        |        |    3.29 |             3.29 | 3.29€/kg    | 3.29€/kg       |     1    | 2025-08-24 |               |
| 10 | RIMI   | [Viščiukų broilerių blauzdelės RIMI,A,1kg](https://www.rimi.lt/e-parduotuve/lt/produktai/mesa-ir-zuvis-/sviezia-paukstiena/vistiena/visciuku-broileriu-blauzdeles-rimi-a-1kg/p/805637)                                                                              |        |    3.89 |             3.89 | 3.89€/kg    | 3.89€/kg       |     1    | 2025-08-24 |               |
|  1 | MAXIMA | [Šviežios viščiukų broilerių blauzdelės WELL DONE, 500 g](https://barbora.lt/produktai/sviezios-visciuku-broileriu-blauzdeles-well-done-500-g-04798)                                                                                                                | -13.1% |    2.29 |             1.99 | 4.58€/kg    | 3.98€/kg       |     0.5  | 2025-08-24 | 2025-08-25    |
|  0 | MAXIMA | [Viščiukų blauzdelės FIESTA su česnakiniu marinatu ir kmynais, 850 g](https://barbora.lt/produktai/visciuku-blauzdeles-fiesta-su-cesnakiniu-marinatu-ir-kmynais-850-g)                                                                                              | -20.0% |    4.49 |             3.59 | 5.28€/kg    | 4.22€/kg       |     0.85 | 2025-08-24 | 2025-08-25    |
|  7 | IKI    | [Šaldytos viščiukų broilerių blauzdelės, ROG](https://www.lastmile.lt/chain/PROMO-CashandCarry/product/Saldytos-visciuku-broileriu-blauzdeles-ROG-V519118)                                                                                                          |        |    4.47 |             4.47 | 4.47€/kg    | 4.47€/kg       |     1    | 2025-08-24 |               |
|  4 | MAXIMA | [Švž. broilerių, užaugintų be antibiotikų, blauzdelės be sąn., 500 g](https://barbora.lt/produktai/svz-broileriu-uzaugintu-be-antibiotiku-blauzdeles-be-san-500-g)                                                                                                  | -23.4% |    2.99 |             2.29 | 5.98€/kg    | 4.58€/kg       |     0.5  | 2025-08-24 | 2025-08-25    |
|  3 | MAXIMA | [Šv. viščiukų broilerių blauzdelės, užauginta be antibiotikų, 1 kg](https://barbora.lt/produktai/sv-visciuku-broileriu-blauzdeles-uzauginta-be-antibiotiku-1-kg)                                                                                                    |        |    4.69 |             4.69 | 4.69€/kg    | 4.69€/kg       |     1    | 2025-08-24 |               |
| 11 | RIMI   | [Švž.viščiukų broilerių blauzd.be ant.RIMI,1kg](https://www.rimi.lt/e-parduotuve/lt/produktai/mesa-ir-zuvis-/sviezia-paukstiena/vistiena/svz-visciuku-broileriu-blauzd-be-ant-rimi-1kg/p/813094)                                                                    |        |    4.79 |             4.79 | 4.79€/kg    | 4.79€/kg       |     1    | 2025-08-24 |               |
| 12 | RIMI   | [Višč. br. blauzdelės be sąn. RIMI, A kl. 500g](https://www.rimi.lt/e-parduotuve/lt/produktai/mesa-ir-zuvis-/sviezia-paukstiena/vistiena/visc-br-blauzdeles-be-san-rimi-a-kl-500g/p/816245)                                                                         |        |    2.69 |             2.69 | 5.38€/kg    | 5.38€/kg       |     0.5  | 2025-08-24 |               |
|  9 | IKI    | [Šviežios viščiukų broilerių blauzdelės VILNIAUS PAUKŠTYNAS, be sąnario, užauginta be antib., 500g](https://www.lastmile.lt/chain/PROMO-CashandCarry/product/Sviezios-visciuku-broileriu-blauzdeles-VILNIAUS-PAUKSTYNAS-be-sanario-uzauginta-be-antib-500g-V509E58) | -13.1% |    3.59 |             3.12 | 7.18€/kg    | 6.24€/kg       |     0.5  | 2025-08-24 | 2025-09-02    |
|  8 | IKI    | [Šviežios viščiukų broilerių blauzdelės VILNIAUS PAUKŠTYNAS, užauginta be antibiotikų, 500 g](https://www.lastmile.lt/chain/PROMO-CashandCarry/product/Sviezios-visciuku-broileriu-blauzdeles-VILNIAUS-PAUKSTYNAS-uzauginta-be-antibiotiku-500-g-V509E57)           |        |    3.47 |             3.47 | 6.94€/kg    | 6.94€/kg       |     0.5  | 2025-08-24 |               |
|  5 | IKI    | [Šviežios viščiukų broilerių blauzdelės IKI MĖSA, 500 g](https://www.lastmile.lt/chain/IKI/product/Sviezios-visciuku-broileriu-blauzdeles-IKI-MESA-500-g-253578)                                                                                                    |        |    3.59 |             3.59 | 7.18€/kg    | 7.18€/kg       |     0.52 | 2025-08-24 |               |

|    | shop   | title                                                                                                                                                                                            | off    |   price |   discount_price | price_per   | discount_per   |   weight | date       | valid_until   |
|---:|:-------|:-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|:-------|--------:|-----------------:|:------------|:---------------|---------:|:-----------|:--------------|
|  5 | RIMI   | [Broilerių filė gabaliukai be antibiotikų,500g](https://www.rimi.lt/e-parduotuve/lt/produktai/mesa-ir-zuvis-/sviezia-paukstiena/vistiena/broileriu-file-gabaliukai-be-antibiotiku-500g/p/971720) | -40.1% |    5.49 |             3.29 | 10.98€/kg   | 6.58€/kg       |     0.5  | 2025-08-24 | 2025-08-25    |
|  1 | MAXIMA | [Lietuviška šviežia viščiukų broilerių filė WELL DONE, 500 g](https://barbora.lt/produktai/lietuviska-sviezia-visciuku-broileriu-file-well-done-500-g)                                           |        |    3.79 |             3.79 | 7.58€/kg    | 7.58€/kg       |     0.5  | 2025-08-24 |               |
|  8 | RIMI   | [Viščiukų krūtinėlių filė RIMI, 500 g](https://www.rimi.lt/e-parduotuve/lt/produktai/mesa-ir-zuvis-/sviezia-paukstiena/vistiena/visciuku-krutineliu-file-rimi-500-g/p/813427)                    |        |    3.99 |             3.99 | 7.98€/kg    | 7.98€/kg       |     0.5  | 2025-08-24 |               |
|  9 | RIMI   | [Švž.viščiukų broilerių filė be  ant.RIMI, 1kg](https://www.rimi.lt/e-parduotuve/lt/produktai/mesa-ir-zuvis-/sviezia-paukstiena/vistiena/svz-visciuku-broileriu-file-be-ant-rimi-1kg/p/813103)   |        |    8.49 |             8.49 | 8.49€/kg    | 8.49€/kg       |     1    | 2025-08-24 |               |
|  4 | IKI    | [Šviežia viščiukų broilerių filė užauginta be antibiotikų, 500 g](https://www.lastmile.lt/chain/IKI/product/Sviezia-visciuku-broileriu-file-uzauginta-be-antibiotiku-500-g-273358)               | -20.0% |    5.99 |             4.79 | 11.98€/kg   | 9.58€/kg       |     0.52 | 2025-08-24 | 2025-08-24    |
|  2 | MAXIMA | [Šviežia viščiukų broilerių filė be antibiotikų WELL DONE, 500 g](https://barbora.lt/produktai/sviez-visciuku-broileriu-file-well-done-uzaug-be-antib-500-g)                                     |        |    4.79 |             4.79 | 9.58€/kg    | 9.58€/kg       |     0.5  | 2025-08-24 |               |
|  0 | MAXIMA | [Šviežia viščiukų broilerių, užaugintų be antibiotikų, filė, 500 g](https://barbora.lt/produktai/sviezia-visciuku-broileriu-uzaugintu-be-antibiotiku-file-500-g)                                 |        |    5.49 |             5.49 | 10.98€/kg   | 10.98€/kg      |     0.5  | 2025-08-24 |               |
| 10 | RIMI   | [Šviež.viščiuko broilerio filė be antib., 500g](https://www.rimi.lt/e-parduotuve/lt/produktai/mesa-ir-zuvis-/sviezia-paukstiena/vistiena/sviez-visciuko-broilerio-file-be-antib-500g/p/995822)   |        |    5.99 |             5.99 | 11.98€/kg   | 11.98€/kg      |     0.5  | 2025-08-24 |               |
|  3 | IKI    | [Šviežia viščiukų broilerių vidinė filė IKI MĖSA, 400 g](https://www.lastmile.lt/chain/IKI/product/Sviezia-visciuku-broileriu-vidine-file-IKI-MESA-400-g-253577)                                 |        |    4.99 |             4.99 | 12.47€/kg   | 12.47€/kg      |     0.42 | 2025-08-24 |               |
|  6 | RIMI   | [Broilerių vidinė filė be antibiotikų, 400 g](https://www.rimi.lt/e-parduotuve/lt/produktai/mesa-ir-zuvis-/sviezia-paukstiena/vistiena/broileriu-vidine-file-be-antibiotiku-400-g/p/971721)      |        |    5.29 |             5.29 | 13.22€/kg   | 13.22€/kg      |     0.4  | 2025-08-24 |               |
|  7 | RIMI   | [Broilerių vidinė filė be antibiotikų, 400 g](https://www.rimi.lt/e-parduotuve/lt/produktai/mesa-ir-zuvis-/sviezia-paukstiena/vistiena/broileriu-vidine-file-be-antibiotiku-400-g/p/971721)      |        |    5.29 |             5.29 | 13.22€/kg   | 13.22€/kg      |     0.4  | 2025-08-24 |               |

|    | shop   | title                                                                                    | off    |   price |   discount_price | price_per   | discount_per   |   weight | date       | valid_until   |
|---:|:-------|:-----------------------------------------------------------------------------------------|:-------|--------:|-----------------:|:------------|:---------------|---------:|:-----------|:--------------|
|  4 | IKI    | [Šilauogės, 500 g.](https://www.lastmile.lt/chain/IKI/product/Silauoges-500-g-254349)    | -47.5% |    5.69 |             2.99 | 11.38€/kg   | 5.98€/kg       |    0.55  | 2025-08-24 | 2025-08-24    |
|  5 | IKI    | [Šilauogės, 250 g](https://www.lastmile.lt/chain/IKI/product/Silauoges-250-g-72479)      | -28.7% |    3.49 |             2.49 | 13.96€/kg   | 9.96€/kg       |    0.26  | 2025-08-24 | 2025-08-24    |
|  0 | MAXIMA | [Lietuviškos šilauogės, 500 g](https://barbora.lt/produktai/lietuviskos-silauoges-500-g) |        |    5.99 |             5.99 | 11.98€/kg   | 11.98€/kg      |    0.5   | 2025-08-24 |               |
|  3 | IKI    | [Šilauogės, 125 g](https://www.lastmile.lt/chain/IKI/product/Silauoges-125-g-2435)       | -20.1% |    1.99 |             1.59 | 15.92€/kg   | 12.72€/kg      |    0.13  | 2025-08-24 | 2025-08-24    |
|  2 | MAXIMA | [Šilauogės kibirėlyje, 400 g](https://barbora.lt/produktai/silauoges-kibirelyje-400-g)   |        |    6.99 |             6.99 | 17.48€/kg   | 17.48€/kg      |    0.4   | 2025-08-24 |               |
|  1 | MAXIMA | [Šilauogės WELL DONE, 125 g](https://barbora.lt/produktai/silauoges-well-done-125-g)     |        |    2.49 |             2.49 | 19.92€/kg   | 19.92€/kg      |    0.125 | 2025-08-24 |               |

|    | shop   | title                                                                                                                                                        | off    |   price |   discount_price | price_per   | discount_per   |   weight | date       | valid_until   |
|---:|:-------|:-------------------------------------------------------------------------------------------------------------------------------------------------------------|:-------|--------:|-----------------:|:------------|:---------------|---------:|:-----------|:--------------|
|  3 | RIMI   | [Špinatai, 200 g](https://www.rimi.lt/e-parduotuve/lt/produktai/vaisiai-darzoves-ir-geles/darzoves-ir-grybai/salotos-ir-ju-misiniai/spinatai-200-g/p/909368) | -23.7% |    1.69 |             1.29 | 8.45€/kg    | 6.45€/kg       |     0.2  | 2025-08-24 | 2025-08-25    |
|  2 | IKI    | [Mažieji špinatai IKI, 100 g](https://www.lastmile.lt/chain/IKI/product/Mazieji-spinatai-IKI-100-g-261481)                                                   | -40.3% |    1.49 |             0.89 | 14.9€/kg    | 8.9€/kg        |     0.11 | 2025-08-24 | 2025-08-24    |
|  1 | MAXIMA | [Mažieji špinatai WELL DONE, 200 g](https://barbora.lt/produktai/mazieji-spinatai-well-done-200-g)                                                           |        |    1.79 |             1.79 | 8.95€/kg    | 8.95€/kg       |     0.2  | 2025-08-24 |               |
|  0 | MAXIMA | [Špinatai WELL DONE, 100 g](https://barbora.lt/produktai/spinatai-well-done-100-g)                                                                           |        |    1.29 |             1.29 | 12.9€/kg    | 12.9€/kg       |     0.1  | 2025-08-24 |               |

### Items that I hunt for discounts to come

In [ ]:
catalogue = collect_catalogue(items=items_hunt, offline=True)
display_catalogue(catalogue)

### Items that I need asap

In [ ]:
catalogue = collect_catalogue(items=items_asap, offline=True)
display_catalogue(catalogue)

### Notes

In [ ]:
* Pievagrybiai Lidl 4€/kg
* Salota (su šaknimis) Lidl 0.89vnt, Maxima 0.99vnt
* nektarinai Lidl 2.49€/kg, persikai Maxima 2.49€/kg
* duona su saulėgrąžomis Lidl 2.48€/kg
* duona su grūdais ir sėklomis Lidl 2.32€/kg
* jogurtas
* burokėliai šaltibarščiams Lidl 2.06€/kg
* majonezas
* menkių filė Lidl 7.99€/kg
* švežia smulkinta kalakutienos krūtinėlė Lidl 6.98€/kg
* šilauogės Lidl 7.49€/kg Maxima 8.38€/kg
* braškės Maxima 6.98€/kg

* varškė -------------------------------------
* braškių uogienė kėdainių
* pjaustyta kiauliena guliašui Maxima 6.98€/kg
* mėsai ar nuolaida?
* colgate
* kiaulienos sprandinė vakumuota 3.99 iš 5.99
* namų kvapai

## Recipe testing

In [8]:
# update these files:
# recipes -> Your Recipe.txt
# ..\cooking -> recipes -> Your Recipe.ipynb
# items.py

recipe_data = parse_recipe(r'recipes\Barščiai.txt')
df = investigate_recipe(recipe_data, offline=False)
display(df)

for ingredient in recipe_data:
    print(f'* {ingredient["header"]}')
    
print('KAINA:', round(df['price'].sum(), 2))
print('NUOLAIDOS:', round(df['discount'].sum(), 2))
print('SVORIS:', round(df.loc[df['units'].isin(['kg', 'l']), 'amount'].sum(), 2))

Please add urls for items: ['Aliejus', 'Pipirai']
RIMI warning: (no price) https://www.rimi.lt/e-parduotuve/lt/produktai/mesa-ir-zuvis-/sviezia-paukstiena/vistiena/broileriu-blauzdeles-be-san-be-ant-vp-500g/p/971711
Done
RIMI warning: (no price) https://www.rimi.lt/e-parduotuve/lt/produktai/vaisiai-darzoves-ir-geles/darzoves-ir-grybai/kopustai/lietuviski-sviezi-baltaguziai-kopustai-kg/p/244676
Done
Done = BULVĖS ...  
Done = BUROKĖLIAI ...  
Done = MORKOS ...  
Done = SVOGŪNAI ...  
RIMI warning: (no price) https://www.rimi.lt/e-parduotuve/lt/produktai/vaisiai-darzoves-ir-geles/darzoves-ir-grybai/paprikos/raudonos-saldziosios-paprikos-70-mm-1kl-1kg/p/269167
Done
Done = POMIDORŲ PASTA ...  
Done = LAURŲ LAPAI ...  
Done = DRUSKA ...  


,amount,units,name,price,discount_price,discount
0,1.00,kg,Viščiukų blauzdelės,3.29,3.29,0.00
1,0.30,kg,Kopūstai,0.36,0.18,0.18
2,0.20,kg,Bulvės,0.11,0.07,0.04
3,0.20,kg,Burokėliai,0.16,0.16,0.00
4,0.12,kg,Morkos,0.08,0.07,0.01
5,0.20,kg,Svogūnai,0.17,0.11,0.06
6,0.20,kg,Paprikos,0.57,0.42,0.15
7,0.02,kg,Pomidorų pasta,0.07,0.07,0.00
8,NaN,None,Aliejus,NaN,NaN,NaN
9,NaN,None,Laurų lapai,NaN,NaN,NaN


* 1 kg vištienos
* 300 g kopūstų
* 200 g bulvių
* 2 burokėliai
* 1 morka
* 1 svogūnas
* 1 paprika
* 1 šaukštas pomidorų pastos
* aliejus
* laurų lapai
* druska
* pipirai
KAINA: 4.81
NUOLAIDOS: 0.44
SVORIS: 2.24
